<a href="https://colab.research.google.com/github/karimNafiz/kaggle_competition_comp432/blob/main/comp432_kaggle_competition_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###All necessary imports

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.modules import linear
from torch.utils.data import TensorDataset, DataLoader, random_split
import math
import matplotlib.pyplot as plt

###Helpers

In [2]:
def count_misclassifications_per_label(model, data_loader, device):
    model.eval()
    classified_dict = {}
    misclassified_dict = {}

    with torch.no_grad():
        for inputs, targets in data_loader:
          outputs = model(inputs)
          _, predicted = torch.max(outputs,1)
          for pred, truth in zip(predicted, targets):
            if pred == truth:
              if pred.item() in classified_dict:
                classified_dict[pred.item()] += 1
              else:
                classified_dict[pred.item()] = 1
            else:
              if pred.item() in misclassified_dict:
                misclassified_dict[pred.item()] += 1
              else:
                misclassified_dict[pred.item()] = 1

    return classified_dict, misclassified_dict



In [3]:
def check_label_ratio(y):
  # convert to np array
  # not doing any error handling don't have the time
  y_np = np.array(y)
  y_count = len(y_np)
  unique, counts = np.unique(y_np, return_counts=True)
  ratio = counts / y_count
  return (unique, ratio)

In [4]:
def get_init_weights_normal(mean, std):
  def init_weights_normal(model):
    if type(model) == nn.Linear:
      nn.init.normal_(model.weight, mean = mean, std = std )
      if model.bias is not None:
        nn.init.constant_(model.bias, 0)
  return init_weights_normal

In [5]:
def print_model_weights(model, num_values=10):
    """
    Prints the weight and bias statistics for each Linear layer in the model.
    num_values: how many values from the weights to show.
    """
    print("\n=== MODEL WEIGHTS SUMMARY ===")

    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            flat_w = module.weight.view(-1)

            print(f"\nLayer: {name} ({module.__class__.__name__})")
            print(f"  Weight shape: {module.weight.shape}")
            print(f"  Bias shape:   {module.bias.shape if module.bias is not None else None}")
            print(f"  First {num_values} weight values: {flat_w[:num_values].tolist()}")
            print(f"  Mean: {float(flat_w.mean()):.6f}, Std: {float(flat_w.std()):.6f}")
            print("-" * 60)

    print("=== END OF WEIGHTS SUMMARY ===\n")

In [6]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    epochs = range(1, len(history["train_loss"]) + 1)

    # ---- Loss Plot ----
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, history["train_loss"], label="Train Loss")
    plt.plot(epochs, history["val_loss"], label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

    # ---- Accuracy Plot ----
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, history["train_acc"], label="Train Accuracy")
    plt.plot(epochs, history["val_acc"], label="Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training vs Validation Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()


In [7]:
import os
import json
from datetime import datetime
import matplotlib.pyplot as plt

def plot_and_save_history(history, out_dir):
    """Plot loss/accuracy and save to files in out_dir."""
    epochs = range(1, len(history["train_loss"]) + 1)

    # ---- Loss Plot ----
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, history["train_loss"], label="Train Loss")
    plt.plot(epochs, history["val_loss"], label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)
    loss_path = os.path.join(out_dir, "loss_curve.png")
    plt.savefig(loss_path, bbox_inches="tight")
    plt.close()

    # ---- Accuracy Plot ----
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, history["train_acc"], label="Train Accuracy")
    plt.plot(epochs, history["val_acc"], label="Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training vs Validation Accuracy")
    plt.legend()
    plt.grid(True)
    acc_path = os.path.join(out_dir, "accuracy_curve.png")
    plt.savefig(acc_path, bbox_inches="tight")
    plt.close()

    return loss_path, acc_path


def log_experiment(
    config: dict,
    history: dict,
    test_results: dict,
    log_root: str = "experiments",
):
    """
    Creates a timestamped folder and logs:
      - config.json       (hyperparameters + setup)
      - metrics.json      (history + test_results)
      - loss_curve.png
      - accuracy_curve.png
    Returns the path to the experiment folder.
    """
    # 1) Make root dir if needed
    os.makedirs(log_root, exist_ok=True)

    # 2) Unique folder name: timestamp + short tag
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    exp_name = f"run_{timestamp}"
    exp_dir = os.path.join(log_root, exp_name)
    os.makedirs(exp_dir, exist_ok=True)

    # 3) Save config (hyperparameters, ratios, etc.)
    config_path = os.path.join(exp_dir, "config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)

    # 4) Save metrics (training history + test)
    metrics = {
        "history": history,
        "test_results": test_results,
    }
    metrics_path = os.path.join(exp_dir, "metrics.json")
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=4)

    # 5) Save plots
    loss_path, acc_path = plot_and_save_history(history, exp_dir)

    print(f"[LOG] Experiment saved to: {exp_dir}")
    print(f"  - Config:   {config_path}")
    print(f"  - Metrics:  {metrics_path}")
    print(f"  - Loss plot: {loss_path}")
    print(f"  - Acc plot:  {acc_path}")

    return exp_dir


###Fully Connected Model


In [124]:
class HouseObjectClassifier(nn.Module):
  def __init__(self, input_features, output_class):
    super().__init__()
    # we have 500 features
    self.linear_layer1 = nn.Linear(input_features, 300)
    self.linear_layer2 = nn.Linear(300, 150)
    # self.linear_layer3 = nn.Linear(256, 128)
    self.output_layer = nn.Linear(150, output_class)

  def forward(self, x):
    x = torch.relu(self.linear_layer1(x))
    x = torch.relu(self.linear_layer2(x))
    # x = torch.relu(self.linear_layer3(x))
    x = self.output_layer(x)
    return x



# I will try them out later

# class HouseObjectClassifier(nn.Module):
#   def __init__(self, input_features, output_class):
#     super().__init__()
#     self.linear_layer1 = nn.Linear(input_features, 512)
#     self.linear_layer2 = nn.Linear(512, 256)
#     self.linear_layer3 = nn.Linear(256, 128)
#     self.output_layer = nn.Linear(128, output_class)

#     self.dropout1 = nn.Dropout(p=0.5)
#     self.dropout2 = nn.Dropout(p=0.5)

#   def forward(self, x):
#     x = torch.relu(self.linear_layer1(x))
#     x = self.dropout1(x)
#     x = torch.relu(self.linear_layer2(x))
#     x = self.dropout2(x)
#     x = torch.relu(self.linear_layer3(x))
#     x = self.output_layer(x)
#     return x

###Function for initializing weights from a normal distribution

In [47]:
def get_init_weights_normal(mean, std):
  def init_weights_normal(model):
    if type(model) == nn.Linear:
      nn.init.normal_(model.weight, mean = mean, std = std )
      if model.bias is not None:
        nn.init.constant_(model.bias, 0)
  return init_weights_normal

###Function for loading the dataset and converting it into tensors

In [48]:
def load_dataset(csv_path: str, drop_columns=None):
    """
    Loads the CSV, drops unwanted columns, and returns:
      X: float tensor of shape (N, num_features)
      y: long tensor of shape (N,)
    Assumes the LAST remaining column is the label.
    """
    if drop_columns is None:
        drop_columns = []

    df = pd.read_csv(csv_path, engine="python", quotechar='"', escapechar='\\')
    df = df.drop(columns=drop_columns, errors="ignore")

    data_np = df.values
    X_np = data_np[:, :-1]
    y_np = data_np[:, -1]

    X = torch.tensor(X_np, dtype=torch.float32)
    y = torch.tensor(y_np, dtype=torch.long)
    return X, y

### Function for creating DataLoaders (required for batching and making life easier)

In [49]:
def create_dataloaders(
    train_X_img, train_y,
    val_X_img, val_y,
    test_X_img, test_y,
    batch_size: int
):
    """
    Wraps tensors in TensorDatasets and DataLoaders.
    """
    train_dataset = TensorDataset(train_X_img, train_y)
    val_dataset   = TensorDataset(val_X_img,   val_y)
    test_dataset  = TensorDataset(test_X_img,  test_y)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

###Functions for normalization

In [50]:
# i might have to re-consider, the normalization strategy
# this function destroyed my model
def compute_normalization(train_X: torch.Tensor):
    """
    Computes feature-wise mean and std from TRAIN data only.
    """
    mean = train_X.mean(dim=0)
    std  = train_X.std(dim=0) + 1e-6   # avoid division by zero
    #print(f"mean {mean} std {std}")
    print(f"mean shape {mean.shape} std shape {std.shape}")
    return mean, std

def log1p_transform(X):
  return torch.log1p(X)



def apply_normalization(X: torch.Tensor, mean: torch.Tensor, std: torch.Tensor):
    """
    Applies (X - mean) / std feature-wise.
    """
    return (X - mean) / std

###Function for splitting the data into train, val and test

In [51]:
# default dict makes my life much easies don't have to think about cases where the key doesn't contain a value
# I realized later this is not important, the data already is very balanced, but better to be safe than sorrow
from collections import defaultdict

def stratified_train_val_test_split(X, y, train_ratio: float, val_ratio: float, test_ratio: float):
    """
    Returns stratified splits of X, y.

    Ensures each class appears in train/val/test according to the given ratios.
    """
    total = train_ratio + val_ratio + test_ratio
    if abs(total - 1.0) > 1e-6:
        raise ValueError("train_ratio + val_ratio + test_ratio must equal 1.")

    # Collect indices for each class
    class_indices = defaultdict(list)
    for idx, label in enumerate(y):
        class_indices[int(label.item())].append(idx)

    train_idx = []
    val_idx = []
    test_idx = []

    # For each class, split indices by ratio
    for cls, indices in class_indices.items():
        indices = torch.tensor(indices)
        indices = indices[torch.randperm(len(indices))]  # shuffle within class

        n = len(indices)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)
        n_test = n - n_train - n_val  # rest goes to test

        train_idx.append(indices[:n_train])
        val_idx.append(indices[n_train:n_train+n_val])
        test_idx.append(indices[n_train+n_val:])

    # Concatenate all classes
    train_idx = torch.cat(train_idx)
    val_idx = torch.cat(val_idx)
    test_idx = torch.cat(test_idx)

    # Shuffle each split (optional but recommended)
    train_idx = train_idx[torch.randperm(len(train_idx))]
    val_idx = val_idx[torch.randperm(len(val_idx))]
    test_idx = test_idx[torch.randperm(len(test_idx))]

    # Gather actual data
    X_train, y_train = X[train_idx], y[train_idx]
    X_val,   y_val   = X[val_idx],   y[val_idx]
    X_test,  y_test  = X[test_idx],  y[test_idx]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

# DEPRICATED
def train_val_test_split(X, y, train_ratio: float, val_ratio: float, test_ratio: float):
    """
    Splits X, y into train/val/test using the provided ratios.
    """
    total = train_ratio + val_ratio + test_ratio
    if total != 1.0:
      raise Exception("train val and test ratio don't add upto 1")


    N = X.size(0)
    indices = torch.randperm(N)

    train_size = int(N * train_ratio)
    val_size   = int(N * val_ratio)
    test_size  = N - train_size - val_size

    train_idx = indices[:train_size]
    val_idx   = indices[train_size:train_size + val_size]
    test_idx  = indices[train_size + val_size:]

    X_train, y_train = X[train_idx], y[train_idx]
    X_val,   y_val   = X[val_idx],   y[val_idx]
    X_test,  y_test  = X[test_idx],  y[test_idx]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


###functions for training

In [52]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in train_loader:
        # inputs: (batch, H, W) -> add channel dim
        #inputs = inputs.unsqueeze(1).to(device)   # (batch, 1, H, W)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # tf is this?
        running_loss += loss.item() * targets.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total   += targets.size(0)

    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [53]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in data_loader:
            #inputs = inputs.unsqueeze(1).to(device)  # (batch, 1, H, W)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item() * targets.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total   += targets.size(0)

    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [54]:
def train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    num_epochs: int,
    device
):
    history = {
        "train_loss": [],
        "train_acc":  [],
        "val_loss":   [],
        "val_acc":    [],
    }

    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss,   val_acc   = evaluate_model(model, val_loader, criterion, device)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch [{epoch}/{num_epochs}] "
            f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
            f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
        )

    return history

###TRAINING

In [90]:
# important constants
TRAIN_CSV_PATH = "drive_karim/MyDrive/kaggle_competition_comp432_datasets/train.csv"
DROP_COLUMNS = ["id"]              # columns to drop from CSV

TRAIN_RATIO = 0.6
VAL_RATIO   = 0.2
TEST_RATIO  = 0.2
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
NUM_CLASSES = 50
MOMENTUM = 0.9
WEIGHT_DECAY = 0
NUM_EPOCHS = 100
log_root="/content/drive_karim/MyDrive/experiments"
# redifing batch size, cuz I am not happy with 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# FOR THE MODEL
INIT_WEIGHTS_MEAN = 0.0
INIT_WEIGHTS_STD = 0.02
IS_INIT_WEIGHTS_NORMAL = True



In [56]:
from google.colab import drive
drive.mount('/content/drive_karim')

Drive already mounted at /content/drive_karim; to attempt to forcibly remount, call drive.mount("/content/drive_karim", force_remount=True).


In [20]:

X, y = load_dataset(TRAIN_CSV_PATH, drop_columns=DROP_COLUMNS)

In [114]:
# @title
train_split, val_split, test_split = stratified_train_val_test_split(X, y, train_ratio=TRAIN_RATIO, val_ratio=VAL_RATIO, test_ratio=TEST_RATIO)
label_ratios = [check_label_ratio(train_split[1]),check_label_ratio(val_split[1]),check_label_ratio(test_split[1])]
for split in label_ratios:
  unique, ratio = split
  for label, ratio in zip(unique,ratio):
    print(f"label {label} ratio {ratio*100}")

label 0 ratio 2.000606682170767
label 1 ratio 1.9977177194528306
label 2 ratio 1.9991622008117986
label 3 ratio 2.000606682170767
label 4 ratio 2.000606682170767
label 5 ratio 2.000606682170767
label 6 ratio 2.002051163529735
label 7 ratio 1.9991622008117986
label 8 ratio 1.9962732380938624
label 9 ratio 2.000606682170767
label 10 ratio 1.9991622008117986
label 11 ratio 1.9991622008117986
label 12 ratio 2.002051163529735
label 13 ratio 1.9991622008117986
label 14 ratio 1.9991622008117986
label 15 ratio 1.9948287567348943
label 16 ratio 1.9977177194528306
label 17 ratio 2.000606682170767
label 18 ratio 2.000606682170767
label 19 ratio 2.000606682170767
label 20 ratio 2.002051163529735
label 21 ratio 1.9991622008117986
label 22 ratio 2.002051163529735
label 23 ratio 2.000606682170767
label 24 ratio 2.000606682170767
label 25 ratio 1.9962732380938624
label 26 ratio 1.9991622008117986
label 27 ratio 2.002051163529735
label 28 ratio 2.000606682170767
label 29 ratio 2.002051163529735
label 3

In [125]:

# create a model instance
model = HouseObjectClassifier(input_features=500, output_class=NUM_CLASSES).to(device)
if(IS_INIT_WEIGHTS_NORMAL):
  # applying closure
  model.apply(get_init_weights_normal(INIT_WEIGHTS_MEAN,INIT_WEIGHTS_STD))
  model.apply(print_model_weights)



=== MODEL WEIGHTS SUMMARY ===

Layer:  (Linear)
  Weight shape: torch.Size([300, 500])
  Bias shape:   torch.Size([300])
  First 10 weight values: [0.04743964225053787, 0.015163221396505833, -0.005545163527131081, -0.02757834643125534, -0.02244647778570652, -0.03231453150510788, -0.004204367287456989, 0.01719009317457676, 0.036971889436244965, -0.018321221694350243]
  Mean: 0.000028, Std: 0.019991
------------------------------------------------------------
=== END OF WEIGHTS SUMMARY ===


=== MODEL WEIGHTS SUMMARY ===

Layer:  (Linear)
  Weight shape: torch.Size([150, 300])
  Bias shape:   torch.Size([150])
  First 10 weight values: [-0.011043897829949856, -0.008502555079758167, -0.01630784012377262, -0.01624302938580513, -0.014607866294682026, 0.021389169618487358, 0.020934993401169777, 0.003310202853754163, -0.016984982416033745, 0.02422950230538845]
  Mean: 0.000008, Std: 0.020052
------------------------------------------------------------
=== END OF WEIGHTS SUMMARY ===


=== MOD

0.001

In [116]:
  # need stochastic gradient descent optimizer because data will be batched
  optimizer = torch.optim.Adam(
      model.parameters(),
      lr=LEARNING_RATE,
      betas=(MOMENTUM, 0.999),
      eps=1e-8,
      weight_decay=WEIGHT_DECAY  # or AdamW with 0.01
  )
  criterion = nn.CrossEntropyLoss()

In [128]:
# always split first
train_X, train_y = train_split
val_X, val_y = val_split
test_X, test_y = test_split

In [129]:
print(f"train_X shape {train_X.shape} train_y shape {train_y.shape}")
print(f"val_X shape {val_X.shape} val_y shape {val_y.shape}")
print(f"test_X shape {test_X.shape} test_y shape {test_y.shape}")

train_X shape torch.Size([69229, 500]) train_y shape torch.Size([69229])
val_X shape torch.Size([23060, 500]) val_y shape torch.Size([23060])
test_X shape torch.Size([23117, 500]) test_y shape torch.Size([23117])


In [130]:
# normliaze
train_X_log1p = log1p_transform(train_X)
val_X_log1p = log1p_transform(val_X)
test_X_log1p = log1p_transform(test_X)

In [131]:
train_loader, val_loader, test_loader = create_dataloaders(train_X_log1p, train_y, val_X_log1p, val_y, test_X_log1p, test_y, BATCH_SIZE)

In [132]:
len(train_loader)

541

In [133]:

def experiment_with_fully_connected(train_loader, val_loader, test_loader, optimizer, loss, NUM_EPOCHS):
  history = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=NUM_EPOCHS, device=device)

  plot_training_history(history)

  # 8. Evaluate on test split
  test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)
  print(f"\nFinal TEST Loss: {test_loss:.4f} | TEST Acc: {test_acc:.4f}")

  # checking what classes are misclassified the most
  classified_dict, misclassified_dict = count_misclassifications_per_label(model, test_loader, device)


  # misclassified_heap = []
  # for key, value in misclassified_dict.items():
  #   heapq.heappush(misclassified_heap, (-value, {key:key, value:value}))

  # classified_heap = []
  # for key, value in classified_dict.items():
  #   heapq.heappush(classified_heap, (-value, {key:key, value:value}))
  # Sorted by count descending
  sorted_misclassified = sorted(
    misclassified_dict.items(), key=lambda x: x[1], reverse=True
  )

  sorted_classified = sorted(
    classified_dict.items(), key=lambda x: x[1], reverse=True
  )



  config = {
    "is_init_weights_normal":IS_INIT_WEIGHTS_NORMAL,
    "mean":INIT_WEIGHTS_MEAN,
    "std":INIT_WEIGHTS_STD,
    "train_ratio": TRAIN_RATIO,
    "val_ratio": VAL_RATIO,
    "test_ratio": TEST_RATIO,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "momentum":MOMENTUM,
    "num_epochs": NUM_EPOCHS,
    "optimizer": optimizer.__class__.__name__,
    "loss_function": criterion.__class__.__name__,
    "num_classes":NUM_CLASSES,
    "model_name": model.__class__.__name__,
    "device": str(device),

    "layers":f"nn.Linear({500},300) -> nn.Relu -> nn.Linear(300,150)->nn.Relu -> nn.Linear(150,50)"
  }

  test_results = {
    "test_loss": test_loss,
    "test_accuracy": test_acc,
    "num_test_samples": int(test_X.shape[0]),
    "classification_count_by_label": [
        {"label": int(k), "count": int(v)} for k, v in sorted_classified
    ],
    "misclassification_count_by_label": [
        {"label": int(k), "count": int(v)} for k, v in sorted_misclassified
    ],

  }

  log_experiment(
    config=config,
    history=history,
    test_results=test_results,
    log_root=log_root  # folder in your drive/colab FS
  )














In [134]:
from google.colab import drive
drive.mount('/content/drive_karim')
experiment_with_fully_connected(train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,optimizer=optimizer, loss=criterion, NUM_EPOCHS=NUM_EPOCHS)

Drive already mounted at /content/drive_karim; to attempt to forcibly remount, call drive.mount("/content/drive_karim", force_remount=True).
Epoch [1/100] Train Loss: 3.9120 | Train Acc: 0.0173 | Val Loss: 3.9120 | Val Acc: 0.0192
Epoch [2/100] Train Loss: 3.9120 | Train Acc: 0.0173 | Val Loss: 3.9120 | Val Acc: 0.0192
Epoch [3/100] Train Loss: 3.9120 | Train Acc: 0.0173 | Val Loss: 3.9120 | Val Acc: 0.0192
Epoch [4/100] Train Loss: 3.9120 | Train Acc: 0.0173 | Val Loss: 3.9120 | Val Acc: 0.0192


KeyboardInterrupt: 